# Apple products

# Data Cleaning

In [21]:
import os
import pandas as pd
#pd.set_option('display.max_rows', None)

df = pd.read_csv(r"C:\Users\valer\Desktop\Projects\Tableu dfs\apple\apple_list.csv")
df.head(10)

,pid,model,price,country_code,country,region,income group,scraped_date
0,0,iPhone 13,"From A$1,199",au,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535
1,1,iPhone SE,From A$679,au,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535
2,2,iPhone 12,From A$999,au,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535
3,3,AirPods(3rd generation),A$279,au,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535
4,4,AirPods Pro,A$399,au,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535
5,5,AirPods(2nd generation),A$219,au,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535
6,6,AirPods Max,A$899,au,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535
7,7,Apple TV 4K,A$249,au,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535
8,8,Apple Watch SE,From A$429,au,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535
9,9,Apple Watch Series 3,From A$299,au,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535


In [22]:
df_rate = pd.read_csv(r"C:\Users\valer\Desktop\Projects\Tableu dfs\apple\rate.csv")

def gen_modif(df):
    '''
    modify general info
    '''
    df_rate.columns = ['time', 'abbrev', 'rate']
    df_rate.loc[15, 'rate'] = 14
    df_rate.loc[15, 'abbrev'] = 'TRY'
    df_rate.loc[16, 'rate'] = 1
    df_rate.loc[16, 'time'] = '2022-01-04'
    df_rate.loc[16, 'abbrev'] = 'USD'
    df_rate.drop('time', axis = 1, inplace = True)
    
    return df

df = gen_modif(df)

In [23]:
list_countries = list(df['country'])
dic = {
    
    'Australia': 'AUD',
    'Austria': 'EUR',
    'Brazil': 'BRL',
    'Canada': 'CAD',
    'Czech Republic': 'CZK',
    'Denmark': 'DKK',
    'Finland': 'EUR',
    'France': 'EUR',
    'Germany': 'EUR',
    'Hungary': 'HUF',
    'India': 'INR',
    'Ireland': 'EUR',
    'Italy': 'EUR',
    'Luxembourg': 'EUR',
    'Mexico': 'MXN',
    'Norway': 'NOK',
    'Philippines': 'PHP',
    'Poland': 'PLN',
    'Portugal': 'EUR',
    'Russia': 'RUB',
    'Spain': 'EUR',
    'Sweden': 'SEK',
    'Thailand': 'THB',
    'Turkey': 'TRY',
    'United States': 'USD'
}

In [24]:
df['money'] = df['country'].map(dic)
df_rate.columns = ['money', 'rate']
df_merged = pd.merge(df, df_rate, on='money', how='inner')

In [26]:
def get_proper_values_for_each_column(x):
    
    '''
    properly encode values in each column
    '''
    
    # spost band
    sports = ['Sport', 'Bracelete', 'Pulseira', 'Pasek', 'Спортивный', 'Spor', 'Correa', 'Urheiluranneke']
    for i in sports:
        if i in x:
            return 'Sport band'
    
    # MacBook
    if '24' in x and 'iMac' in x or 'iMac²' in x:
        return '24-inch iMac²'
    
    # AirPods
    elif ('AirPods' in x and '2' in x) or ('AirPods' in x and 'segunda' in x) or ('AirPods' in x and 'seconda' in x) \
    or ('AirPods' in x and 'segunda' in x) or ('AirPods' in x and 'andra' in x):
        return 'AirPods(2nd generation)'
    
    elif ('AirPods' in x and '3' in x) or ('AirPods' in x and 'tredje' in x) or ('AirPods' in x and 'tercera' in x) \
    or ('AirPods' in x and 'terza' in x):
        return 'AirPods(3rd generation)'
    
    # AirPensil
    elif ('Apple Pencil' in x and '2' in x) or ('Apple Pencil' in x and 'seconda' in x) \
    or ('Apple Pencil' in x and 'segunda' in x) or ('Apple Pencil' in x and 'andra' in x) \
    or ('Apple Pencil' in x and 'második' in x):
        return 'Apple Pencil (2nd generation)'
    
    # Magic Mouse
    elif 'Magic Mouse' in x:
        return 'Magic Mouse'
    
    # Cards
    elif ('iTunes' in x) or ('Carta' in x) or ('Tarjeta' in x) or ('iTunes' in x):
        return 'Card iTunes'
           
    else:
        return x
    
    
    
# apply functions
df_merged['model_cleaned'] = df_merged['model'].apply(lambda x: get_proper_values_for_each_column(x))
df_merged['price'] = df_merged['price'].str.replace(' ', '')
df_merged['price'] = df_merged['price'].str.extract(r'(\d+\,?\.?\d+)')
df_merged['price_initial'] = df_merged['price']
df_merged['price'] = df_merged['price'].astype(str)

In [18]:
def digits_price(x):
    '''gets the money num'''
    if '.' in x and len(x) <= 6:
        x = x.replace('.', ',')
        return x
    else:
        return x
    
    
def digits_1_price(x):
    '''gets the money num'''
    if ',' in x:
        pre, post = x.split(',')
        if len(post) == 2:

            x = x.replace(',' ,'.')
            return x
        else:
            return x
    else:
        return x

In [27]:
# apply functions
df_merged['price'] = df_merged['price'].apply(lambda x: digits_price(x))
df_merged['price'] = df_merged['price'].apply(lambda x: digits_1_price(x))
df_merged['price']  = df_merged['price'].str.replace(',', '')
df_merged['price'] = pd.to_numeric(df_merged['price'], errors = 'coerce')
df_merged['price_usd'] = df_merged['price']/df_merged['rate']
df_merged = df_merged[['model_cleaned', 'price_usd', 'country']]
df_merged['min_price'] = df_merged.groupby(['model_cleaned', 'country'])['price_usd'].transform('min')
df_merged = df_merged[['model_cleaned', 'min_price', 'country']]

In [28]:
df_merged.to_excel(r"C:\Users\valer\Desktop\Projects\Tableu dfs\apple\df.xlsx")

# Cleaned dataset

In [29]:
df_merged.head(20)

,model_cleaned,min_price,country
0,iPhone 13,864.960756,Australia
1,iPhone SE,489.831821,Australia
2,iPhone 12,720.680397,Australia
3,AirPods(3rd generation),201.271102,Australia
4,AirPods Pro,287.839318,Australia
5,AirPods(2nd generation),157.986994,Australia
6,AirPods Max,648.540217,Australia
7,Apple TV 4K,179.629048,Australia
8,Apple Watch SE,309.481372,Australia
9,Apple Watch Series 3,215.699138,Australia
